<a href="https://colab.research.google.com/github/SaraBabakN/Model-Search/blob/master/Skip_Connection_4_0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from datetime import datetime
import torch
import torchvision
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


train_set = torchvision.datasets.CIFAR10('./dataset', train=True, download=True,
                                         transform=transforms.Compose([transforms.ToTensor()]))


def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

0it [00:00, ?it/s]

 99%|█████████▉| 168484864/170498071 [00:11<00:00, 16874607.78it/s]

Extracting ./dataset/cifar-10-python.tar.gz to ./dataset


In [0]:
lr = 0.01
batch_size = 200
dataset_size = 50000
epoch_num = 40
cor_ths = 0.5
data_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)

In [0]:
# sns.set()    
# plt.figure(i)
#     ax = sns.heatmap(
#       corr, 
#       vmin=-1, vmax=1, center=0,
#       cmap=sns.color_palette("hls", 10),
#       square=True
#     )
#     plt.title({i})

# plt.show 

In [0]:
class myModel(nn.Module):

    def __init__(self, block,groups=1, width_per_group=64):
        super(myModel, self).__init__()
        norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer
        self.inplanes = 64
        self.dilation = 1
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3,64, kernel_size=7, stride=2, padding=3,bias=False)
        self.bn1 = norm_layer(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64)
        self.layer2 = self._make_layer(block, 64)
        self.layer3 = self._make_layer(block, 128, stride=2)
        self.layer4 = self._make_layer(block, 128,)
        self.layer5 = self._make_layer(block, 256, stride=2)
        self.layer6 = self._make_layer(block, 256)
        self.layer7 = self._make_layer(block, 512, stride=2)
        self.layer8 = self._make_layer(block, 512)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 , 10)

    def _make_layer(self, block, planes, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes, stride),
                norm_layer(planes),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes
        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.layer7(x)
        x = self.layer8(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)
class BasicBlock(nn.Module): 
    expansion = 1 
    __constants__ = ['downsample']

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        norm_layer = nn.BatchNorm2d
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out    

def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)

def myResnet(pretrained=False, progress=True, **kwargs):
    return myModel(BasicBlock,**kwargs)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
myNet = torch.load("/content/drive/My Drive/data1/myNetV4.pth")

In [0]:
output= []
for counter in range (0,63):
    first = myNet.conv1.weight[counter][0].clone().detach().cpu()
    for i in range (counter + 1 ,64):
      second = myNet.conv1.weight[i][0].clone().detach().cpu()
      corr = np.corrcoef(first, second)
      for index in range(7):
        if (abs(corr[index][7+index])< cor_ths):
          break
        if index== 6 :
          output.append([counter , i ])
for x in output: 
    first = myNet.conv1.weight[x[0]][1].clone().detach().cpu()
    second = myNet.conv1.weight[x[1]][1].clone().detach().cpu()
    corr = np.corrcoef(first, second)
    for index in range(7):
      if (abs(corr[index][7+index])< cor_ths):
        break
      if index== 6 :
        print (x)
# stopped at 1 

In [0]:
# 1 ---> 5 
# 2 ---> 6 
# 3 ---> 6 
# 4 ---> 6
# 5 ---> 6 
# 6 ---> 7
# 7 ---> 9
modulList = list(myNet.layer8.modules())
for x in modulList: 
  print(x)
  print("%%%%%%")

Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)
%%%%%%
BasicBlock(
  (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)
%%%%%%
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
%%%%%%
BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, t

In [0]:
modulList = list(myNet.layer8.modules())
convLayer = modulList[5]
output= []
lenght = 3
num_filter = 512  
for counter in range (0,num_filter - 1 ):
    first = convLayer.weight[counter][0].clone().detach().cpu()
    for i in range (counter + 1 , num_filter):
      second = convLayer.weight[i][0].clone().detach().cpu()
      corr = np.corrcoef(first, second)
      for index in range(lenght):
        if (abs(corr[index][lenght+index])< cor_ths):
          break
        if index== lenght - 1  :
          output.append([counter , i ])
print(len(output))
for j in range (1,10): 
  output1 = [] 
  for x in output: 
    first = convLayer.weight[x[0]][2*j - 1].clone().detach().cpu()
    second = convLayer.weight[x[1]][2*j - 1].clone().detach().cpu()
    corr = np.corrcoef(first, second)
    for index in range(lenght):
      if (abs(corr[index][lenght+index])< cor_ths):
        break
      if index == lenght-1 :
        output1.append(x)
  print(len(output1))
  output = []
  for x in output1: 
      first = convLayer.weight[x[0]][2*j].clone().detach().cpu()
      second = convLayer.weight[x[1]][2*j].clone().detach().cpu()
      corr = np.corrcoef(first, second)
      for index in range(lenght):
        if (abs(corr[index][lenght+index])< cor_ths):
          break
        if index == lenght-1 :
          output.append(x)
  print(len(output))


53951
22845
9813
4149
1699
694
261
127
32
17
7
2
2
0
0
0
0
0
0


In [0]:
# Whole layer 
a = torch.norm(x1.reshape(batch_size,-1).clone().detach().cpu(),dim=1)
b = torch.norm(x2.reshape(batch_size,-1).clone().detach().cpu(),dim=1)
c = torch.norm((l3).reshape(batch_size,-1).clone().detach().cpu(),dim=1)
t= range (batch_size)
plt.figure(0)
plt.plot(t,a[t] , 'ro' , label = "x[t]" )
plt.plot(t,b[t] , 'b*' , label = "x[t-1]")
plt.plot(t,c[t] , 'gs' , label = "relu(x[t]+x[t-1])")
plt.xlabel("image")
plt.ylabel("norm of all filters")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title("raw output of each layer")

In [0]:
a = torch.norm(x1.clone().detach().cpu(),dim=(2,3))
b = torch.norm(x2.clone().detach().cpu(),dim=(2,3))
c = torch.norm(x3.clone().detach().cpu(),dim=(2,3))
d = torch.norm(x4.clone().detach().cpu(),dim=(2,3))
indic_a = (torch.nonzero( a[0] < 0.0001)).reshape(-1)
indic_b = (torch.nonzero( b[0] < 0.0001)).reshape(-1)
indic_c = (torch.nonzero( c[0] < 0.0001)).reshape(-1)
indic_d = (torch.nonzero( d[0] < 0.0001)).reshape(-1)
for i in range(200): 
  indic_a = (torch.nonzero( a[i][indic_a] < 0.0001)).reshape(-1)
  indic_b = (torch.nonzero( b[i][indic_b] < 0.0001)).reshape(-1)
  indic_c = (torch.nonzero( c[i][indic_c] < 0.0001)).reshape(-1)
  indic_d = (torch.nonzero( d[i][indic_d] < 0.0001)).reshape(-1)
print(indic_a)
print(indic_b)
print(indic_c)
print(indic_d)
